# Mathematical Underpinnings - Lab 6

In [1]:
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

## Useful functions

In [2]:
def discetize_2bins(X):
    X_discrete = 1*(X >= 0)
    return X_discrete

In [3]:
def conditional_permutation(X, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    X_b = np.zeros(n)

    for i in range(n_z_values):

        z_value_tmp = z_values[i]

        X_b[Z == z_value_tmp] = np.random.permutation(X[Z == z_value_tmp])

    return X_b

In [4]:
def conditional_mutual_information(X, Y, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):

        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition)/n

        cmi += p_z*mi_XY_z

    return cmi

In [5]:
# II(X;Y;Z)
def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

In [6]:
# II(X;Y;Z1;Z2)
def interaction_information2(X, Y, Z1, Z2):
    Z_1_and_2 = 2*Z2 + Z1
    return interaction_information(X, Y, Z_1_and_2) - interaction_information(X, Y, Z1) - interaction_information(X, Y, Z2)

## Task 1

In [15]:
def secmi2(X, Y, Z):
    ii = 0
    for i in range(Z.shape[1]):
        ii += interaction_information(X, Y, Z[:, i])
    return ii + mutual_info_score(X, Y)

def secmi3(X,Y,Z):
    ii = secmi2(X, Y, Z)
    ii2 = 0
    for i in range(Z.shape[1]):
        for j in range(i + 1, Z.shape[1]):
            ii2 += interaction_information2(X, Y, Z[:, i], Z[:, j])
    return ii + ii2

### a)

In [8]:
def cond_indep_test_permutation(X, Y, Z, B, stat):

    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))

    if stat == "cmi":
        stat_value = conditional_mutual_information(X, Y, Z_1dim)
    if stat == "secmi2":
        stat_value = secmi2(X, Y, Z)
    if stat == "secmi3":
        stat_value = secmi3(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        X_b = conditional_permutation(X, Z_1dim)

        if stat == "cmi":
            stat_value_b = conditional_mutual_information(X_b, Y, Z_1dim)
        if stat == "secmi2":
            stat_value_b = secmi2(X_b, Y, Z)
        if stat == "secmi3":
            stat_value_b = secmi3(X_b, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

### b)

In [9]:
def generate_data_B(n = 100):
    Y = np.random.normal(0, 1, n)
    Y = discetize_2bins(Y)

    Z1 = np.random.normal(2 * Y - 1, 1, n)
    Z1 = discetize_2bins(Z1)

    Z2 = np.random.normal(2 * Y - 1, 1, n) 
    Z2 = discetize_2bins(Z2)

    Z3 = np.random.normal(2 * Y - 1, 1, n)
    Z3 = discetize_2bins(Z3)

    X = np.random.normal(2 * Z1 - 1, 1, n)   
    X = discetize_2bins(X)
    return X, Y, Z1, Z2, Z3

In [18]:
# Z1 + Z2
N = 100
results_case_1 = {'cmi': 0, 'secmi2': 0, 'secmi3': 0}
for i in range(N):
    X, Y, Z1, Z2, Z3 = generate_data_B()
    for stat in ["cmi", "secmi2", "secmi3"]:
        stat_value, p_val = cond_indep_test_permutation(X, Y, np.transpose([Z1, Z2]), 50, stat)
        if p_val < 0.05:
            results_case_1[stat] += 1

for stat in ["cmi", "secmi2", "secmi3"]:
    print("### ", stat.upper())
    print("Accepted: ", N - results_case_1[stat])
    print("Rejected: ", results_case_1[stat], "\n")


###  CMI
Accepted:  99
Rejected:  1 

###  SECMI2
Accepted:  100
Rejected:  0 

###  SECMI3
Accepted:  100
Rejected:  0 



In [19]:
# Z2 + Z3
N = 100
results_case_2 = {'cmi': 0, 'secmi2': 0, 'secmi3': 0}
for i in range(N):
    X, Y, Z1, Z2, Z3 = generate_data_B()
    for stat in ["cmi", "secmi2", "secmi3"]:
        stat_value, p_val = cond_indep_test_permutation(X, Y, np.transpose([Z2, Z3]), 50, stat)
        if p_val < 0.05:
            results_case_2[stat] += 1

for stat in ["cmi", "secmi2", "secmi3"]:
    print("### ", stat.upper())
    print("Accepted: ", N - results_case_2[stat])
    print("Rejected: ", results_case_2[stat], "\n")

###  CMI
Accepted:  2
Rejected:  98 

###  SECMI2
Accepted:  1
Rejected:  99 

###  SECMI3
Accepted:  1
Rejected:  99 



### c)

In [20]:
def generate_data_C(n = 100):
    X = np.random.randint(0, 2, n)
    Z1 = np.random.randint(0, 2, n)
    Z2 = np.random.randint(0, 2, n)
    Z3 = np.random.randint(0, 2, n)
    Y = np.zeros(n)

    for i in range(n):
        if (X[i] + Z1[i] + Z2[i]) % 2 == 1:
            Y[i] = np.random.choice([0, 1], p=[0.2, 0.8])
        else:
            Y[i] = np.random.choice([0, 1], p=[0.8, 0.2])
    return X, Y, Z1, Z2, Z3

In [21]:
# Z1 + Z2
N = 100
results_case_1 = {'cmi': 0, 'secmi2': 0, 'secmi3': 0}
for i in range(N):
    X, Y, Z1, Z2, Z3 = generate_data_C()
    for stat in ["cmi", "secmi2", "secmi3"]:
        stat_value, p_val = cond_indep_test_permutation(X, Y, np.transpose([Z1, Z2]), 50, stat)
        if p_val < 0.05:
            results_case_1[stat] += 1

for stat in ["cmi", "secmi2", "secmi3"]:
    print("### ", stat.upper())
    print("Accepted: ", N - results_case_1[stat])
    print("Rejected: ", results_case_1[stat], "\n")

###  CMI
Accepted:  0
Rejected:  100 

###  SECMI2
Accepted:  95
Rejected:  5 

###  SECMI3
Accepted:  0
Rejected:  100 



In [22]:
# Z2 + Z3
N = 100
results_case_1 = {'cmi': 0, 'secmi2': 0, 'secmi3': 0}
for i in range(N):
    X, Y, Z1, Z2, Z3 = generate_data_C()
    for stat in ["cmi", "secmi2", "secmi3"]:
        stat_value, p_val = cond_indep_test_permutation(X, Y, np.transpose([Z2, Z3]), 50, stat)
        if p_val < 0.05:
            results_case_1[stat] += 1

for stat in ["cmi", "secmi2", "secmi3"]:
    print("### ", stat.upper())
    print("Accepted: ", N - results_case_1[stat])
    print("Rejected: ", results_case_1[stat], "\n")

###  CMI
Accepted:  96
Rejected:  4 

###  SECMI2
Accepted:  99
Rejected:  1 

###  SECMI3
Accepted:  95
Rejected:  5 



## Task 2
 
in R